In [1]:
using AutomotiveDrivingModels
using ProfileView

include(Pkg.dir("AutomotiveDrivingModels", "src", "io", "filesystem_utils.jl"))
const INCLUDE_FILE = Pkg.dir("AutomotiveDrivingModels", "scripts", "extract_params.jl")
include(INCLUDE_FILE)

type DataSubset
    name::AbstractString
    behavior_to_match::UInt16
    behavior_to_avoid::UInt16
end

features = union(collect(allfeatures()), INDICATOR_SET2)

filters = AbstractFeature[Feature_IsClean{:f_accel}(),
                          Feature_IsClean{:f_des_angle}()]

runlog_extract_params = RunLogSegmentExtractParameters(SIM_HORIZON_IN_FRAMES, SIM_HISTORY_IN_FRAMES,
                                                       FRAMESKIP_BETWEEN_EXTRACTED_SCENES, PDSET_FRAMES_PER_SIM_FRAME)

dataset_extract_params = DatasetExtractParams(ContextClass.NULL, ContextClass.NULL, features, runlog_extract_params,
                                              filters=filters)

runlog_filepaths = filter!(s->splitext(s)[2] == ".jld", readdir(RUNLOG_DIR))
for i in 1 : length(runlog_filepaths)
    runlog_filepaths[i] = joinpath(RUNLOG_DIR, runlog_filepaths[i])
end

INFO: Precompiling module AutomotiveDrivingModels...


In [2]:
subset = DataSubset("freeflow", ContextClass.FREEFLOW, ContextClass.NULL)

dataset_extract_params.behavior_to_match = subset.behavior_to_match
dataset_extract_params.behavior_to_avoid = subset.behavior_to_avoid

model_training_data = pull_model_training_data(dataset_extract_params, runlog_filepaths)
nothing

tot_n_frames: 37780
pass: 37780
elapsed time: 48.501056723 seconds


In [3]:
n_samples_monte_carlo_integration = 10

srand(1)
@time pull_model_training_data(dataset_extract_params, runlog_filepaths)
nothing

tot_n_frames: 37780
pass: 37780
 52.670897 seconds (834.07 M allocations: 17.439 GB, 13.73% gc time)


In [4]:
srand(1)
Profile.clear()
@profile pull_model_training_data(dataset_extract_params, runlog_filepaths)


pass: 37780
tot_n_frames: 37780

LoadError: LoadError: TypeError: isa: expected Type{T}, got AutomotiveDrivingModels.Features.Feature_PosFyaw
while loading In[4], in expression starting on line 3


pass: 37780
